# Analyse der Internetnutzung und Schülerleistung

In dieser Analyse untersuchen wir den Zusammenhang zwischen der Internetnutzung und der schulischen Leistung von Schülern. Wir betrachten, wie die Nutzung des Internets das Lernverhalten, die Motivation und die Ergebnisse der Schüler beeinflusst.

Durch die Analyse von Daten zur Internetnutzung und schulischen Leistung können wir wichtige Erkenntnisse gewinnen, die uns helfen, Bildungsstrategien zu verbessern und den Schülern dabei zu helfen, ihr volles Potenzial auszuschöpfen.

Wir werden verschiedene Variablen betrachten, wie z.B. die Zeit, die Schüler online verbringen, die Art der Internetnutzung (z.B. für schulische Zwecke oder Freizeitaktivitäten) und die Auswirkungen auf ihre schulischen Leistungen. Darüber hinaus werden wir auch demografische und sozioökonomische Faktoren berücksichtigen, um ein umfassendes Bild zu erhalten.

Zu Beginn werden die Daten geladen und eine explorative Datenanalyse durchgeführt, um die Verteilung der Variablen zu untersuchen und mögliche Zusammenhänge zu identifizieren. Anschließend werden wir verschiedene statistische Methoden anwenden, um den Zusammenhang zwischen Internetnutzung und Schülerleistung zu untersuchen.

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import statsmodels.api as sm

# Load the data
pisa_math = pd.read_csv('pisaMathe.csv', index_col='Jahr', parse_dates=['Jahr'])
pisa_nature = pd.read_csv('pisaNatur.csv', index_col='Jahr', parse_dates=['Jahr'])
pisa_reading = pd.read_csv('pisaLese.csv', index_col='Jahr', parse_dates=['Jahr'])

internet_usage = pd.read_csv('internetNutzung.csv', index_col='Jahr', parse_dates=['Jahr']).xs('InternetMinuten', drop_level=False, axis=1)
print_usage = pd.read_csv('printNutzung.csv', index_col='Jahr', parse_dates=['Jahr'])
politics = pd.read_csv('politischesInteresse.csv', index_col='Jahr', parse_dates=['Jahr'])
music_school_attendance = pd.read_csv('schulerMusikschulen.csv', index_col='Jahr', parse_dates=['Jahr'])
music_schools = pd.read_csv('musikSchulenAnzahl.csv', index_col='Jahr', parse_dates=['Jahr'])

In [2]:
# Merge the data
data = pd.concat([pisa_math, pisa_nature, pisa_reading, internet_usage, print_usage, politics, music_school_attendance, music_schools], axis=1).sort_index().ffill().bfill()
data = data.loc['2000':]
data = data / data.iloc[0,:]

In [3]:
# plot pisa
fig = px.line(data, y=['Mathematik', 'Naturwissenschaften', 'Lesekompetenz'])
fig.update_layout(title='PISA Ergebnisse 2000-2024')
fig.update_yaxes(title='Punkte')
fig.show()

In [4]:
# plot activities
fig = px.line(data, y=['InternetMinuten',
       'Bücher', 'Zeitschriften_Magazine', 'Tageszeitung',
       'PolitikInteresse(prozent)', 'AnzahlSchuler'])
fig.update_layout(title='Aktivitäten 2000-2024')
fig.update_yaxes(title='Werte')
fig.show()

In [5]:
# fit a linear model
data['score'] = (data['Mathematik'] + data['Naturwissenschaften'] + data['Lesekompetenz']) / 3
X = data[['InternetMinuten', 'Bücher', 'Zeitschriften_Magazine', 'Tageszeitung', 'PolitikInteresse(prozent)', 'AnzahlSchuler']]
X = sm.add_constant(X)
y = data['score']

model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  score   R-squared:                       0.660
Model:                            OLS   Adj. R-squared:                  0.540
Method:                 Least Squares   F-statistic:                     5.498
Date:                Mon, 01 Jul 2024   Prob (F-statistic):            0.00252
Time:                        15:59:21   Log-Likelihood:                 71.639
No. Observations:                  24   AIC:                            -129.3
Df Residuals:                      17   BIC:                            -121.0
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
=============================================================================================
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                         0.7467      0.149      5.018      0.000       0.433       1.061
InternetMinuten               0.0451      0.024      1.845      0.082      -0.006       0.097
Bücher                        0.0222      0.071      0.314      0.757      -0.127       0.171
Zeitschriften_Magazine       -0.0570      0.109     -0.521      0.609      -0.287       0.174
Tageszeitung                  0.1791      0.069      2.579      0.020       0.033       0.326
PolitikInteresse(prozent)     0.0886      0.063      1.417      0.175      -0.043       0.221
AnzahlSchuler                -0.0034      0.045     -0.074      0.942      -0.099       0.092
==============================================================================
Omnibus:                        0.550   Durbin-Watson:                   1.790
Prob(Omnibus):                  0.760   Jarque-Bera (JB):                0.627
Skew:                           0.144   Prob(JB):                        0.731
Kurtosis:                       2.262   Cond. No.                         196.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""